In [3]:
import os
import sys
import pandas as pd
from tqdm import tqdm # Sử dụng tqdm thông thường cho script
import logging
import matplotlib.pyplot as plt # Import matplotlib
import seaborn as sns # Import seaborn

In [4]:

print("\n=== 1. Tải Mã Nguồn và Các Thư Viện Cần Thiết ===")
# Thiết lập logging cơ bản
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Điều chỉnh đường dẫn để import các module từ thư mục src
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.append(project_root)

try:
    from src.gameLogic.game import Game
    from src.gameLogic.board import SquareType
    from src.gameLogic.player import Player, PlayerType
    from src.ai.minimax.evaluator import StateEvaluator, HeuristicType
    from src.ai.ML_DL_models.model import Model as ML_Model
except ImportError as e:
    print(f"Lỗi Import: {e}. Đảm bảo bạn đang chạy script từ thư mục gốc của dự án hoặc sys.path đã đúng.")
    exit()


=== 1. Tải Mã Nguồn và Các Thư Viện Cần Thiết ===


c:\Users\THANH TAN\Desktop\HT\242\IntroductionToAI\BTL\BTL2\BTL2_AI-introduce\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# --- Phần 2: Khởi Tạo Các Agent ---
print("\n=== 2. Khởi Tạo Các Agent ===")
agents = {}

# Tải ML Model dùng chung (LSTM, GPT-2, RF, XGB)
logging.info("Đang tải các ML models...")
shared_ml_model_loader = ML_Model()
logging.info("Các ML models đã được tải (hoặc cố gắng tải).")

# Định nghĩa weights tối ưu cho Minimax
minimax_optimal_weights = {
    HeuristicType.DISC_DIFF: 5/60,
    HeuristicType.MOBILITY: 15/60,
    HeuristicType.CORNERS: 40/60
}
optimal_state_evaluator = StateEvaluator(weights=minimax_optimal_weights)

# Agent 0: Random
agents['agent_0_Random'] = {
    'black': Player(PlayerType.RANDOM, SquareType.BLACK),
    'white': Player(PlayerType.RANDOM, SquareType.WHITE)
}

# Agent 1: Minimax (Depth 1)
agents['agent_1_Minimax_D1'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, optimal_state_evaluator, depth=1),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, optimal_state_evaluator, depth=1)
}

# Agent 2: Minimax (Depth 2)
agents['agent_2_Minimax_D2'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, optimal_state_evaluator, depth=2),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, optimal_state_evaluator, depth=2)
}

# Agent 3: Minimax (Depth 3)
agents['agent_3_Minimax_D3'] = {
    'black': Player(PlayerType.MINIMAX, SquareType.BLACK, optimal_state_evaluator, depth=3),
    'white': Player(PlayerType.MINIMAX, SquareType.WHITE, optimal_state_evaluator, depth=3)
}

# Agent 4: Random Forest
if shared_ml_model_loader.rf_model:
    agents['agent_4_RF'] = {
        'black': Player(PlayerType.RF_MODEL, SquareType.BLACK, ml_model_instance=shared_ml_model_loader),
        'white': Player(PlayerType.RF_MODEL, SquareType.WHITE, ml_model_instance=shared_ml_model_loader)
    }
else:
    logging.warning("RF Model không khả dụng, Agent 4 sẽ là Random.")
    agents['agent_4_RF_Fallback_Random'] = {
        'black': Player(PlayerType.RANDOM, SquareType.BLACK),
        'white': Player(PlayerType.RANDOM, SquareType.WHITE)
    }


# Agent 5: XGBoost
if shared_ml_model_loader.xgb_model:
    agents['agent_5_XGB'] = {
        'black': Player(PlayerType.XGB_MODEL, SquareType.BLACK, ml_model_instance=shared_ml_model_loader),
        'white': Player(PlayerType.XGB_MODEL, SquareType.WHITE, ml_model_instance=shared_ml_model_loader)
    }
else:
    logging.warning("XGBoost Model không khả dụng, Agent 5 sẽ là Random.")
    agents['agent_5_XGB_Fallback_Random'] = {
         'black': Player(PlayerType.RANDOM, SquareType.BLACK),
        'white': Player(PlayerType.RANDOM, SquareType.WHITE)
    }

# Agent 6: LSTM
if shared_ml_model_loader.lstm:
    agents['agent_6_LSTM'] = {
        'black': Player(PlayerType.LSTM, SquareType.BLACK, ml_model_instance=shared_ml_model_loader),
        'white': Player(PlayerType.LSTM, SquareType.WHITE, ml_model_instance=shared_ml_model_loader)
    }
else:
    logging.warning("LSTM Model không khả dụng, Agent 6 sẽ là Random.")
    agents['agent_6_LSTM_Fallback_Random'] = {
         'black': Player(PlayerType.RANDOM, SquareType.BLACK),
        'white': Player(PlayerType.RANDOM, SquareType.WHITE)
    }

# Agent 7: GPT-2
if shared_ml_model_loader.gpt2 and shared_ml_model_loader.tokenizer:
    agents['agent_7_GPT2'] = {
        'black': Player(PlayerType.GPT2, SquareType.BLACK, ml_model_instance=shared_ml_model_loader),
        'white': Player(PlayerType.GPT2, SquareType.WHITE, ml_model_instance=shared_ml_model_loader)
    }
else:
    logging.warning("GPT-2 Model không khả dụng, Agent 7 sẽ là Random.")
    agents['agent_7_GPT2_Fallback_Random'] = {
         'black': Player(PlayerType.RANDOM, SquareType.BLACK),
        'white': Player(PlayerType.RANDOM, SquareType.WHITE)
    }

print(f"Đã khởi tạo {len(agents)} agents: {list(agents.keys())}")

2025-05-19 21:44:15,328 - INFO - Đang tải các ML models...



=== 2. Khởi Tạo Các Agent ===


c:\Users\THANH TAN\Desktop\HT\242\IntroductionToAI\BTL\BTL2\BTL2_AI-introduce\venv\Lib\site-packages\keras\src\layers\layer.py:940: UserWarning: Layer 'embedding_1' (of type Embedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
2025-05-19 21:44:30,791 - INFO - Các ML models đã được tải (hoặc cố gắng tải).


Đã khởi tạo 8 agents: ['agent_0_Random', 'agent_1_Minimax_D1', 'agent_2_Minimax_D2', 'agent_3_Minimax_D3', 'agent_4_RF', 'agent_5_XGB', 'agent_6_LSTM', 'agent_7_GPT2']


In [6]:
# --- Phần 3: Hàm Mô Phỏng Trận Đấu ---
print("\n=== 3. Định Nghĩa Hàm Mô Phỏng Trận Đấu ===")
def simulate_match(agent_black_name, agent_white_name, current_agents_dict):
    agent_black_config = current_agents_dict.get(agent_black_name)
    agent_white_config = current_agents_dict.get(agent_white_name)
    
    if not agent_black_config or not agent_white_config:
        logging.error(f"Không tìm thấy cấu hình cho agent: {agent_black_name} hoặc {agent_white_name}")
        return None

    player_black = agent_black_config["black"]
    player_white = agent_white_config["white"]
    
    game = Game(player_black, player_white)
    game.move_history_str = "" 
    
    turn_count = 0
    max_turns = 120 

    while not game.is_finished and turn_count < max_turns:
        game.get_player_move()
        
        if game.next_move is None:
            game.prev_move = None 
            game.change_turn()
            game.update_valid_moves() 
            if not game.is_any_valid_move_on_board(): 
                game.is_finished = True
        else:
            game.make_move()
            game.change_turn()
            game.update_valid_moves()
            game.update_scores()
        
        game.check_finished() 
        turn_count += 1
        
    if turn_count >= max_turns and not game.is_finished:
        logging.warning(f"Trận đấu {agent_black_name} vs {agent_white_name} đạt tối đa số lượt. Tự động xác định người thắng.")
        game.is_finished = True 
        if not game.game_result: game.determine_winner()
    elif not game.game_result and game.is_finished: 
        game.determine_winner()

    return {
        'game_result': game.game_result,
        'black_score': game.black_score,
        'white_score': game.white_score,
        'agent_black': agent_black_name,
        'agent_white': agent_white_name,
        'total_moves': len(game.move_history_str) // 2
    }


=== 3. Định Nghĩa Hàm Mô Phỏng Trận Đấu ===


In [7]:
# --- Phần 4: Tạo Kết Quả Giải Đấu ---
print("\n=== 4. Tạo Kết Quả Giải Đấu ===")
def generate_tournament_results(agent_dict_param):
    match_results_list = []
    agent_names_list = list(agent_dict_param.keys())
    num_agents = len(agent_names_list)

    for i in tqdm(range(num_agents), desc="Đang thi đấu các cặp Agent"):
        for j in range(num_agents):
            if i == j: 
                continue
            
            agent1_name = agent_names_list[i]
            agent2_name = agent_names_list[j]
            
            logging.info(f"Đang thi đấu: {agent1_name} (Đen) vs {agent2_name} (Trắng)")
            result = simulate_match(agent1_name, agent2_name, agent_dict_param) # Truyền agents vào
            if result:
                match_results_list.append(result)
            else:
                logging.error(f"Không thể mô phỏng trận đấu giữa {agent1_name} và {agent2_name}")
    return match_results_list

tournament_match_results = generate_tournament_results(agents) # Truyền dict agents hiện tại



=== 4. Tạo Kết Quả Giải Đấu ===


Đang thi đấu các cặp Agent:   0%|          | 0/8 [00:00<?, ?it/s]2025-05-19 21:44:30,969 - INFO - Đang thi đấu: agent_0_Random (Đen) vs agent_1_Minimax_D1 (Trắng)
2025-05-19 21:44:31,194 - INFO - Đang thi đấu: agent_0_Random (Đen) vs agent_2_Minimax_D2 (Trắng)
2025-05-19 21:44:31,927 - INFO - Đang thi đấu: agent_0_Random (Đen) vs agent_3_Minimax_D3 (Trắng)
2025-05-19 21:44:40,031 - INFO - Đang thi đấu: agent_0_Random (Đen) vs agent_4_RF (Trắng)
2025-05-19 21:44:59,783 - INFO - Đang thi đấu: agent_0_Random (Đen) vs agent_5_XGB (Trắng)
2025-05-19 21:45:03,971 - INFO - Đang thi đấu: agent_0_Random (Đen) vs agent_6_LSTM (Trắng)
c:\Users\THANH TAN\Desktop\HT\242\IntroductionToAI\BTL\BTL2\BTL2_AI-introduce\venv\Lib\site-packages\keras\src\layers\layer.py:940: UserWarning: Layer 'embedding_1' (of type Embedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.


In [9]:
# --- Phần 5: Xử Lý Kết Quả và Tạo Bảng Xếp Hạng ---
print("\n=== 5. Xử Lý Kết Quả và Tạo Bảng Xếp Hạng ===")
df_results = pd.DataFrame(tournament_match_results)

output_dir = os.getcwd()
results_csv_path = os.path.join(output_dir, "grand_tournament_match_results.csv")
df_results.to_csv(results_csv_path, index=False)
print(f"\nKết quả chi tiết các trận đấu đã được lưu vào '{results_csv_path}'")
print("5 trận đầu tiên:")
print(df_results.head())

all_agent_names_played = pd.concat([df_results['agent_black'], df_results['agent_white']]).unique()
df_league = pd.DataFrame(index=all_agent_names_played)

df_league['Wins'] = 0
df_league['Draws'] = 0
df_league['Losses'] = 0
df_league['Points'] = 0
df_league['Games_Played'] = 0
df_league['Goals_For'] = 0
df_league['Goals_Against'] = 0

for index, row in df_results.iterrows():
    black_agent = row['agent_black']
    white_agent = row['agent_white']
    
    if black_agent not in df_league.index: df_league.loc[black_agent] = 0
    if white_agent not in df_league.index: df_league.loc[white_agent] = 0
        
    df_league.loc[black_agent, 'Games_Played'] += 1
    df_league.loc[white_agent, 'Games_Played'] += 1
    df_league.loc[black_agent, 'Goals_For'] += row['black_score']
    df_league.loc[black_agent, 'Goals_Against'] += row['white_score']
    df_league.loc[white_agent, 'Goals_For'] += row['white_score']
    df_league.loc[white_agent, 'Goals_Against'] += row['black_score']
    
    if row['game_result'] == "Black Wins":
        df_league.loc[black_agent, 'Wins'] += 1
        df_league.loc[black_agent, 'Points'] += 3
        df_league.loc[white_agent, 'Losses'] += 1
    elif row['game_result'] == "White Wins":
        df_league.loc[white_agent, 'Wins'] += 1
        df_league.loc[white_agent, 'Points'] += 3
        df_league.loc[black_agent, 'Losses'] += 1
    elif row['game_result'] == "Draw":
        df_league.loc[black_agent, 'Draws'] += 1
        df_league.loc[black_agent, 'Points'] += 1
        df_league.loc[white_agent, 'Draws'] += 1
        df_league.loc[white_agent, 'Points'] += 1

df_league['Goal_Difference'] = df_league['Goals_For'] - df_league['Goals_Against']
df_league_sorted = df_league.sort_values(by=['Points', 'Goal_Difference', 'Goals_For'], ascending=[False, False, False])

league_csv_path = os.path.join(output_dir, "grand_tournament_league_table.csv")
df_league_sorted.to_csv(league_csv_path)
print(f"\nBảng xếp hạng giải đấu đã được lưu vào '{league_csv_path}'")
print("\nBảng xếp hạng:")
print(df_league_sorted)


=== 5. Xử Lý Kết Quả và Tạo Bảng Xếp Hạng ===

Kết quả chi tiết các trận đấu đã được lưu vào 'c:\Users\THANH TAN\Desktop\HT\242\IntroductionToAI\BTL\BTL2\BTL2_AI-introduce\src\experiments\grand_tournament_match_results.csv'
5 trận đầu tiên:
  game_result  black_score  white_score     agent_black         agent_white  \
0  White Wins           13           51  agent_0_Random  agent_1_Minimax_D1   
1  White Wins           10           54  agent_0_Random  agent_2_Minimax_D2   
2  White Wins           10           54  agent_0_Random  agent_3_Minimax_D3   
3        Draw           32           32  agent_0_Random          agent_4_RF   
4  White Wins           22           42  agent_0_Random         agent_5_XGB   

   total_moves  
0           60  
1           60  
2           60  
3           60  
4           60  

Bảng xếp hạng giải đấu đã được lưu vào 'c:\Users\THANH TAN\Desktop\HT\242\IntroductionToAI\BTL\BTL2\BTL2_AI-introduce\src\experiments\grand_tournament_league_table.csv'

Bảng xếp h

## Nhận Xét Kết Quả Giải Đấu (Tập trung vào RF, GPT-2, XGBoost, LSTM)

Dựa trên bảng xếp hạng và kết quả chi tiết các trận đấu:

1.  **Minimax Vượt Trội:** Các agent Minimax (đặc biệt D3, D1, D2) chiếm ưu thế rõ rệt, cho thấy hiệu quả của thuật toán tìm kiếm kết hợp hàm đánh giá tốt.

2.  **GPT-2 (`agent_7_GPT2`) - Model ML/DL Hứa Hẹn Nhất:**
    *   Xếp hạng 4 toàn giải, cao nhất trong nhóm ML/DL với 5 thắng, 1 hòa (16 điểm).
    *   Có khả năng đánh bại các model ML/DL khác (RF, XGB) và cả Random agent.
    *   Thua các Minimax agent, cho thấy còn khoảng cách về chiều sâu chiến thuật.
    *   *Tiềm năng:* Finetune thêm, prompt engineering, hoặc kết hợp với search nông.

3.  **XGBoost (`agent_5_XGB`) - Hiệu Suất Khá:**
    *   Xếp hạng 5 với 5 thắng (15 điểm), tốt hơn RF và LSTM.
    *   Thắng được Random, RF, LSTM nhưng thua các Minimax và GPT-2.
    *   Hiệu số bàn thắng bại khá âm, gợi ý có thể thua với cách biệt lớn.
    *   *Tiềm năng:* Feature engineering (mobility, kiểm soát cạnh,...) và tinh chỉnh siêu tham số có thể cải thiện đáng kể.

4.  **Random Forest (`agent_4_RF`) - Cần Cải Thiện Nhiều:**
    *   Xếp hạng 7 với 3 thắng, 1 hòa (10 điểm).
    *   Hiệu suất chưa vượt trội so với Random agent và thua đậm nhiều trận.
    *   *Tiềm năng:* Giống như XGBoost, RF rất cần feature engineering để nắm bắt các khía cạnh chiến thuật của Othello thay vì chỉ dựa vào trạng thái thô của bàn cờ.

5.  **LSTM (`agent_6_LSTM`) - Kết Quả Thấp Nhất:**
    *   Đứng cuối bảng (không tính Random) với 2 thắng, 1 hòa (7 điểm).
    *   Có vẻ mô hình hiện tại chưa học được các chiến lược hiệu quả.
    *   *Tiềm năng:* Cần xem xét lại kiến trúc model, cách biểu diễn dữ liệu chuỗi, và chất lượng/số lượng dữ liệu huấn luyện.

**Kết Luận Chung cho ML/DL:**

*   Các model ML/DL (đặc biệt là GPT-2) cho thấy khả năng học được một số chiến thuật cơ bản và có thể đánh bại các agent yếu hơn.
*   Tuy nhiên, để cạnh tranh với Minimax ở độ sâu trung bình, việc **feature engineering (cho RF/XGB)** và **cải tiến kiến trúc/dữ liệu huấn luyện (cho LSTM/GPT-2)** là rất quan trọng.
*   Việc chỉ sử dụng trạng thái bàn cờ thô (64 ô) làm input có thể chưa đủ để các model này phát huy hết tiềm năng.